In [8]:
import cloudscraper
import time
from bs4 import BeautifulSoup, Comment
from io import StringIO
import pandas as pd

ModuleNotFoundError: No module named 'cloudscraper'

In [3]:
url = "https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats"